In [ ]:
# !pip install datasets
# !pip install accelerate -U

In [ ]:
from datasets import load_dataset

# Load Movie Review Dataset

In [ ]:
df = load_dataset('rotten_tomatoes')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
df['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [ ]:
df['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [ ]:
df['train']['text'][40:50]

['newton draws our attention like a magnet , and acts circles around her better known co-star , mark wahlberg .',
 "the story loses its bite in a last-minute happy ending that's even less plausible than the rest of the picture . much of the way , though , this is a refreshingly novel ride .",
 'fuller would surely have called this gutsy and at times exhilarating movie a great yarn .',
 "'compleja e intelectualmente retadora , el ladrón de orquídeas es uno de esos filmes que vale la pena ver precisamente por su originalidad . '",
 'the film makes a strong case for the importance of the musicians in creating the motown sound .',
 'karmen moves like rhythm itself , her lips chanting to the beat , her long , braided hair doing little to wipe away the jeweled beads of sweat .',
 'gosling provides an amazing performance that dwarfs everything else in the film .',
 "a real movie , about real people , that gives us a rare glimpse into a culture most of us don't know .",
 'tender yet lacerating

In [ ]:
df['train']['label'][40:50]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Tokenize text

In [ ]:
from transformers import AutoTokenizer

myTokenizer = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(myTokenizer)

In [ ]:
def tokenize(inp):
  return tokenizer(inp['text'], padding='max_length', truncation=True)

In [ ]:
tk_df = df.map(tokenize, batched=True)

In [ ]:
tk_df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})

# Convert to Pytorch datasets


In [ ]:
# Convert datasets to PyTorch datasets
train_dataset = tk_df['train']
val_dataset = tk_df['validation']
test_dataset = tk_df['test']

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8530
})

# Init model for classification

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification

myModel = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(myModel, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Define Training Args

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./rottenResults',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./rottenLogs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Init Trainer

In [ ]:
from transformers import Trainer, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Define compute_metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    acc = accuracy_score(labels, pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define Optimizer to use
optimizer = AdamW(model.parameters(), lr=5e-5)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=(optimizer, None),
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_result = trainer.train(); train_result

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.473900,0.362875,0.839587,0.835419,0.857708,0.814259
2,0.292700,0.417444,0.841463,0.847885,0.814879,0.883677
3,0.138000,0.648914,0.837711,0.837253,0.839623,0.834897


TrainOutput(global_step=1602, training_loss=0.2896607460898258, metrics={'train_runtime': 682.3795, 'train_samples_per_second': 37.501, 'train_steps_per_second': 2.348, 'total_flos': 3389840731607040.0, 'train_loss': 0.2896607460898258, 'epoch': 3.0})

# Eval model on Val dataset

In [ ]:
eval_result = trainer.evaluate()
# print(eval_result)

print("Eval Accuracy:", eval_result['eval_accuracy'])
print("Eval F1-score:", eval_result['eval_f1'])
print("Eval Precision:", eval_result['eval_precision'])
print("Eval Recall:", eval_result['eval_recall'])

Eval Accuracy: 0.8395872420262664
Eval F1-score: 0.8354186717998076
Eval Precision: 0.857707509881423
Eval Recall: 0.8142589118198874


# Eval model on Test dataset

In [ ]:
test_results = trainer.evaluate(test_dataset)
# print(trest_results)

print("Test Accuracy:", test_results['eval_accuracy'])
print("Test F1-score:", test_results['eval_f1'])
print("Test Precision:", test_results['eval_precision'])
print("Test Recall:", test_results['eval_recall'])

Test Accuracy: 0.8367729831144465
Test F1-score: 0.8326923076923077
Test Precision: 0.854043392504931
Test Recall: 0.8123827392120075
